In [2]:
import json

from aiohttp import ClientSession
from web3 import AsyncWeb3, AsyncHTTPProvider

ENDPOINT_URI = "https://twilight-side-general.quiknode.pro/687fadb1927609e28aa8e868360673178405f233/"

class AsyncEthClient:
    def __init__(self, endpoint_uri: str, request_kwargs=None):
        self.endpoint_uri = endpoint_uri
        self.request_kwargs = request_kwargs or {}
        self.session = None
        self.w3 = None

    async def connect(self):
        """Khởi tạo session + provider + web3 (chỉ làm 1 lần)."""
        if self.session is None:
            self.session = ClientSession()

        provider = AsyncHTTPProvider(self.endpoint_uri, request_kwargs=self.request_kwargs)
        self.w3 = AsyncWeb3(provider)

        # Reuse session — quan trọng!
        await self.w3.provider.cache_async_session(self.session)

    async def close(self):
        """Đóng session khi kết thúc."""
        if self.session and not self.session.closed:
            await self.session.close()

client = AsyncEthClient(ENDPOINT_URI, request_kwargs={"ssl": False})
await client.connect()

In [5]:
latest = await client.w3.eth.block_number
print("Latest Block Number:", latest)

block = await client.w3.eth.get_block(latest, full_transactions=True)
block_json = json.loads(AsyncWeb3.to_json(block))
print(json.dumps(block_json, indent=4))

Latest Block Number: 23995736
{
    "baseFeePerGas": 91212083,
    "blobGasUsed": 393216,
    "difficulty": 0,
    "excessBlobGas": 129798503,
    "extraData": "0xe29ca82051756173617220287175617361722e77696e2920e29ca8",
    "gasLimit": 60000000,
    "gasUsed": 12424267,
    "hash": "0xac9e0b0728f5f22d39fd91b32ee80b5cb679a2e4b917c4e8d699e584efa4c022",
    "logsBloom": "0xa9f9b578937a9b7e6ab1bad1e5ed9f80c3a622c8452b4141dca5d72260138d2bd27c2564a1cd6072055ec34e8483c994aeb56a30dfa7ef7f58e7a4f9313c5167c6aede5f38ae5628fbb4f17adf43c8e6ebd934d810c5497c86d40b36d3274aab4063414f6f643f40bd9f36876a6dbf99321510eb3bde14cd5aabd096999b3c74ba1fadfb313887b80fcb3d7a5b4b602390c061372348f2dab23865d2bc740f9baf4a5c20e88c20acc2f214e134eab4285cf0558c82436e4c6d4dbb6254157d4185817e1b2c45d92b29e989bc6ba9bc1e9f6c5f4f74fc529dae4deb6e80f0ef12b354edf3ab79ae8a64f141eaaae178c5e5fbb926d73d52d84691b2a373cc676d",
    "miner": "0x396343362be2A4dA1cE0C1C210945346fb82Aa49",
    "mixHash": "0xe5499d6179b53aed3eedf7ff43106bbf12e

In [4]:
tx_hashes = block["transactions"][:5]
# print(tx_hashes)

tx_hashes_json = json.loads(AsyncWeb3.to_json(tx_hashes))
print(json.dumps(tx_hashes_json, indent=4))

[
    "0x72729ab3c44f80dad07442427e397d88fdadb50204a3a9b26f332214fed5eb2e",
    "0x2f02803633919276ab171c041d16d48af7dab1e718859f5bd364edb68bf32bda",
    "0xbeefa51b542b42f37377df6a163146c21dfda34dd529cd6eafa5fd4ec7bacfa8",
    "0x5f9833b5b5bda36c0d6cf7950b703a6faed76ee7bad2d9f7ad7f3b90af3b3d15",
    "0xd0f54d0067ac40c3a580392a52cfe1c8750c0d8ea098ed785b4c4027bdee74e6"
]


In [4]:
receipts = await client.w3.provider.make_request(
    "eth_getBlockReceipts",
    [hex(latest)],
)

# In số lượng receipts
print("Total receipts:", len(receipts["result"]))

# In chỉ 1 receipt đầu tiên
print("First receipt:", json.dumps(receipts["result"][0], indent=4)[:2000])  # cắt bớt

Total receipts: 204
First receipt: {
    "blockHash": "0x87f8a987371a78510865e9c960099f6c105ee5784984570678d298c37a2b6013",
    "blockNumber": "0x16e24e9",
    "contractAddress": null,
    "cumulativeGasUsed": "0x28ea5",
    "effectiveGasPrice": "0x259644f66",
    "from": "0x44d187839a08594448d3929679f3a324d00207b2",
    "gasUsed": "0x28ea5",
    "logs": [
        {
            "address": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
            "topics": [
                "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c",
                "0x00000000000000000000000080a64c6d7f12c47b7c66c5b4e20e72bc1fcd5d9e"
            ],
            "data": "0x00000000000000000000000000000000000000000000000000b1a2bc2ec50000",
            "blockNumber": "0x16e24e9",
            "transactionHash": "0x5eb6a97bcf05ed6b85db17a8f38a31208715d53dd1936609fac01b7a14f6d7c6",
            "transactionIndex": "0x0",
            "blockHash": "0x87f8a987371a78510865e9c960099f6c105ee5784984570678d298

In [ ]:
# tx_receipt = await w3.eth.get_transaction_receipt(tx_hash)
#
# print(json.dumps(json.loads(AsyncWeb3.to_json(tx_receipt)), indent=4))

In [ ]:
receipts = await client.w3.provider.make_request(
    "eth_getBlockReceipts",
    [hex(latest)],
)

# In số lượng receipts
print("Total receipts:", len(receipts["result"]))

# In chỉ 1 receipt đầu tiên
print("First receipt:", json.dumps(receipts["result"][0], indent=4)[:2000])

In [5]:
import json
import asyncio

# ---- ABI cơ bản ----
ERC20_ABI = json.loads("""
[
    {"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},
    {"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},
    {"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"}
]
""")

ERC721_ABI = json.loads("""
[
    {"constant":true,"inputs":[{"name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"}
]
""")

ERC721_INTERFACE_ID = "0x80ac58cd"
TRANSFER_TOPIC = client.w3.keccak(text="Transfer(address,address,uint256)").hex()

print(f"Transfer Topic: {TRANSFER_TOPIC}")

Transfer Topic: ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef


In [7]:
import asyncio
import json
from web3 import AsyncWeb3
from aiohttp.client_exceptions import ClientResponseError
from web3.exceptions import BadFunctionCallOutput, ContractLogicError
from typing import Dict, List, Optional

# Constants
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

ERC721_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "interfaceID", "type": "bytes4"}],
        "name": "supportsInterface",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_tokenId", "type": "uint256"}],
        "name": "ownerOf",
        "outputs": [{"name": "owner", "type": "address"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

ERC721_INTERFACE_ID = "0x80ac58cd"  # ERC721 interface ID
TRANSFER_TOPIC = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"  # Transfer event topic

# Global cache for token types
token_cache = {}

# Semaphore to limit concurrent requests
MAX_CONCURRENT_REQUESTS = 2
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)


# Improved retry/backoff helper with better error handling
async def call_with_retry(func, retries=3, delay=1):
    for i in range(retries):
        try:
            result = await func()
            return result
        except (ClientResponseError, TimeoutError, asyncio.CancelledError, OSError) as e:
            if hasattr(e, 'status') and e.status == 429:  # Rate limit
                print(f"Rate limited, waiting {delay}s before retry {i+1}/{retries}")
                await asyncio.sleep(delay)
                delay *= 2  # exponential backoff
            elif isinstance(e, (TimeoutError, asyncio.CancelledError, OSError)):
                print(f"Timeout or connection error, waiting {delay}s before retry {i+1}/{retries}")
                await asyncio.sleep(delay)
                delay *= 2
            else:
                print(f"Other error: {e}")
                raise
        except (BadFunctionCallOutput, ContractLogicError, ValueError) as e:
            print(f"Contract error: {e}")
            # return None if call fails due to contract not supporting the function
            return None
        except Exception as e:
            print(f"Unexpected error in call_with_retry: {e}")
            if i < retries - 1:
                await asyncio.sleep(delay)
                delay *= 2
            else:
                return None
    return None


# Improved token type detection with better error handling
async def detect_token_type(token_address: str):
    if token_address in token_cache:
        return token_cache[token_address]

    async with semaphore:
        try:
            # Try ERC20
            erc20_contract = client.w3.eth.contract(address=token_address, abi=ERC20_ABI)
            decimals = await call_with_retry(erc20_contract.functions.decimals().call)
            if decimals is not None:
                token_cache[token_address] = "ERC20"
                return "ERC20"

            # Try ERC721
            erc721_contract = client.w3.eth.contract(address=token_address, abi=ERC721_ABI)
            try:
                is_erc721 = await call_with_retry(lambda: erc721_contract.functions.supportsInterface(ERC721_INTERFACE_ID).call())
                if is_erc721:
                    token_cache[token_address] = "ERC721"
                    return "ERC721"
            except Exception:
                # If supportsInterface fails, try another ERC721 method
                try:
                    # Just try to call a function that exists in ERC721 but not ERC20
                    token_name = await call_with_retry(erc721_contract.functions.name().call)
                    if token_name is not None:
                        token_cache[token_address] = "ERC721"
                        return "ERC721"
                except Exception:
                    pass

            # If none of the above work, mark as unknown
            token_cache[token_address] = "UNKNOWN"
            return "UNKNOWN"
        except Exception as e:
            print(f"Error detecting token type for {token_address}: {e}")
            token_cache[token_address] = "UNKNOWN"
            return "UNKNOWN"


# Improved transfer extraction with fixed data handling
async def extract_transfers_from_block(block_number: int):
    logs = await client.w3.eth.get_logs({
        "fromBlock": block_number,
        "toBlock": block_number,
        "topics": [TRANSFER_TOPIC]
    })

    transfers = []

    # Get unique token addresses to detect types in parallel
    unique_tokens = list({log["address"] for log in logs})

    # Create tasks to detect token types with proper error handling
    token_type_tasks = {}
    for addr in unique_tokens:
        token_type_tasks[addr] = asyncio.create_task(detect_token_type(addr))

    # Wait for all tasks to complete with exception handling
    results = await asyncio.gather(*token_type_tasks.values(), return_exceptions=True)

    # Process results and update cache
    for addr, result in zip(unique_tokens, results):
        if isinstance(result, Exception):
            print(f"Error detecting token type for {addr}: {result}")
            token_cache[addr] = "UNKNOWN"
        else:
            token_cache[addr] = result

    # Process logs and create transfer records
    for log in logs:
        token_address = log["address"]
        token_type = token_cache.get(token_address, "UNKNOWN") # Lấy loại token trước để quyết định cách parse

        # Parse địa chỉ From/To (Topic 1 và 2)
        # Lưu ý: Cần kiểm tra độ dài topics để tránh lỗi index out of range
        if len(log["topics"]) < 3:
            continue

        from_addr = "0x" + log["topics"][1].hex()[-40:]
        to_addr = "0x" + log["topics"][2].hex()[-40:]

        value = 0

        # LOGIC QUAN TRỌNG:
        if token_type == "ERC721":
            # Nếu là NFT, Token ID nằm ở Topic 3
            if len(log["topics"]) >= 4:
                value = int(log["topics"][3].hex(), 16)
            else:
                value = 0 # Trường hợp không chuẩn
        else:
            # Nếu là ERC20 hoặc Unknown, lấy Value từ Data
            data = log["data"]
            if isinstance(data, bytes):
                data_hex = data.hex()
            elif isinstance(data, str):
                data_hex = data[2:] if data.startswith("0x") else data
            else:
                data_hex = ""

            try:
                value = int(data_hex, 16) if data_hex else 0
            except ValueError:
                value = 0

        transfers.append({
            "token": token_address,
            "from": from_addr,
            "to": to_addr,
            "value": value, # Với ERC721 đây là TokenID, với ERC20 đây là Amount
            "tx_hash": log["transactionHash"].hex(),
            "block_number": block_number,
            "token_type": token_type
        })

    return transfers

transfers = await extract_transfers_from_block(latest)

print(json.dumps(json.loads(AsyncWeb3.to_json(transfers[:1])), indent=2))

[
  {
    "token": "0xdAC17F958D2ee523a2206206994597C13D831ec7",
    "from": "0xba12222222228d8ba445958a75a0704d566bf2c8",
    "to": "0x252344dd5be86df28a6eb7efb1e3b97fd7d4b127",
    "value": 12000000000,
    "tx_hash": "2f02803633919276ab171c041d16d48af7dab1e718859f5bd364edb68bf32bda",
    "block_number": 23994985,
    "token_type": "ERC20"
  }
]


In [9]:
print(f"{'Token Address':<42} | {'Token ID':<15} | {'Transaction Hash'}")
print("-" * 100)

found_nft = False

for tx in transfers:
    # Chỉ xử lý nếu token_type là ERC721
    if tx.get("token_type") == "ERC721":
        found_nft = True
        token_address = tx['token']
        token_id = tx['value']  # Lưu ý: Code trích xuất phải lấy từ topic[3] mới đúng TokenID
        tx_hash = tx['tx_hash']

        print(f"{token_address:<42} | {token_id:<15} | {tx_hash}")

if not found_nft:
    print("Không tìm thấy token ERC-721 nào trong danh sách.")

Token Address                              | Token ID        | Transaction Hash
----------------------------------------------------------------------------------------------------
Không tìm thấy token ERC-721 nào trong danh sách.


In [21]:
logs = await client.w3.eth.get_logs({
    "fromBlock": latest,
    "toBlock": latest,
    "topics": [TRANSFER_TOPIC]
})

t = json.loads(AsyncWeb3.to_json(logs[0]))

print(json.dumps(t, indent=4))

{
    "address": "0xdAC17F958D2ee523a2206206994597C13D831ec7",
    "topics": [
        "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef",
        "0x000000000000000000000000ba12222222228d8ba445958a75a0704d566bf2c8",
        "0x000000000000000000000000252344dd5be86df28a6eb7efb1e3b97fd7d4b127"
    ],
    "data": "0x00000000000000000000000000000000000000000000000000000002cb417800",
    "blockNumber": 23994985,
    "transactionHash": "0x2f02803633919276ab171c041d16d48af7dab1e718859f5bd364edb68bf32bda",
    "transactionIndex": 1,
    "blockHash": "0xd16d72c3445eac0221a84d08ef36e5bfb9d3c920dad5a9bf08aedea07a67d3df",
    "blockTimestamp": "0x693bc00f",
    "logIndex": 0,
    "removed": false
}


In [13]:
# 2. Định nghĩa ABI tối thiểu (Chỉ cần hàm decimals)
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# 3. Địa chỉ USDT trên Ethereum (Token mẫu để test)
USDT_ADDRESS = "0xdAC17F958D2ee523a2206206994597C13D831ec7"

async def debug_contract_steps():
    print(f"--- BƯỚC 1: KHỞI TẠO ĐỐI TƯỢNG CONTRACT ---")
    
    # Dòng code bạn thắc mắc
    erc20_contract = client.w3.eth.contract(address=USDT_ADDRESS, abi=ERC20_ABI)
    
    # In ra xem nó là cái gì
    print(f"1. Biến 'erc20_contract' là kiểu dữ liệu gì?")
    print(f"   => {type(erc20_contract)}")
    print(f"2. Nội dung bên trong nó trông như thế nào?")
    print(f"   => {erc20_contract}")
    print(f"3. Nó có chứa hàm 'decimals' không?")
    print(f"   => {erc20_contract.functions.decimals}")
    print("\n(Lúc này CHƯA có kết nối mạng nào được thực hiện, chỉ là tạo object trong Python)\n")

    print(f"-" * 50)
    print(f"--- BƯỚC 2: GỌI HÀM DECIMALS().CALL() ---")
    
    # Kiểm tra kết nối trước
    if await client.w3.is_connected():
        print(">> Đã kết nối Node, đang gửi yêu cầu đọc dữ liệu...")
        
        # Dòng code thứ 2 bạn thắc mắc
        # call() thực hiện việc đọc dữ liệu từ blockchain (read-only)
        decimals_value = await erc20_contract.functions.decimals().call()
        
        print(f"1. Giá trị trả về của 'decimals_value':")
        print(f"   => {decimals_value}")
        print(f"2. Kiểu dữ liệu:")
        print(f"   => {type(decimals_value)}")
        
        print("\n(Lúc này code ĐÃ gửi request lên Node và nhận về con số 6 - vì USDT có 6 số thập phân)")
    else:
        print("Kết nối mạng thất bại.")

# Chạy hàm debug
await debug_contract_steps()

--- BƯỚC 1: KHỞI TẠO ĐỐI TƯỢNG CONTRACT ---
1. Biến 'erc20_contract' là kiểu dữ liệu gì?
   => <class 'web3._utils.datatypes.AsyncContract'>
2. Nội dung bên trong nó trông như thế nào?
   => <web3._utils.datatypes.AsyncContract object at 0x111aeb0a0>
3. Nó có chứa hàm 'decimals' không?
   => <Function decimals()>

(Lúc này CHƯA có kết nối mạng nào được thực hiện, chỉ là tạo object trong Python)

--------------------------------------------------
--- BƯỚC 2: GỌI HÀM DECIMALS().CALL() ---
>> Đã kết nối Node, đang gửi yêu cầu đọc dữ liệu...
1. Giá trị trả về của 'decimals_value':
   => 6
2. Kiểu dữ liệu:
   => <class 'int'>

(Lúc này code ĐÃ gửi request lên Node và nhận về con số 6 - vì USDT có 6 số thập phân)


In [14]:
import asyncio
from web3 import AsyncWeb3, AsyncHTTPProvider

# --- CẤU HÌNH CỦA BẠN ---
ERC721_INTERFACE_ID = "0x80ac58cd" # Đây là Magic Number của chuẩn ERC-721

ERC721_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "interfaceID", "type": "bytes4"}],
        "name": "supportsInterface",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

# --- ĐỊA CHỈ TEST ---
# 1. Bored Ape Yacht Club (Đây là NFT chuẩn ERC-721)
BAYC_ADDRESS = "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"

# 2. USDT (Đây là ERC-20, KHÔNG phải NFT -> Để so sánh đối chứng)
USDT_ADDRESS = "0xdAC17F958D2ee523a2206206994597C13D831ec7"

async def check_is_nft():
    if not await client.w3.is_connected():
        print("Không thể kết nối node")
        return

    print(f"{'Token Name':<10} | {'Address':<42} | {'Là ERC-721?'}")
    print("-" * 70)

    # --- TEST 1: Kiểm tra BAYC (NFT) ---
    try:
        nft_contract = client.w3.eth.contract(address=BAYC_ADDRESS, abi=ERC721_ABI)
        # Gọi hàm supportsInterface với ID của ERC721
        is_bayc_nft = await nft_contract.functions.supportsInterface(ERC721_INTERFACE_ID).call()
        print(f"{'BAYC':<10} | {BAYC_ADDRESS} | {is_bayc_nft} (Đúng)")
    except Exception as e:
        print(f"BAYC Error: {e}")

    # --- TEST 2: Kiểm tra USDT (ERC-20) ---
    # USDT cũ không hỗ trợ ERC-165 (hàm supportsInterface), nên gọi hàm này thường sẽ lỗi hoặc False
    try:
        usdt_contract = client.w3.eth.contract(address=USDT_ADDRESS, abi=ERC721_ABI)
        is_usdt_nft = await usdt_contract.functions.supportsInterface(ERC721_INTERFACE_ID).call()
        print(f"{'USDT':<10} | {USDT_ADDRESS} | {is_usdt_nft} (Sai)")
    except Exception:
        # Hầu hết ERC20 sẽ nhảy vào đây vì không có hàm supportsInterface
        print(f"{'USDT':<10} | {USDT_ADDRESS} | False (Không hỗ trợ hàm này -> Không phải NFT)")

await check_is_nft()

Token Name | Address                                    | Là ERC-721?
----------------------------------------------------------------------
BAYC       | 0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D | True (Đúng)
USDT       | 0xdAC17F958D2ee523a2206206994597C13D831ec7 | False (Không hỗ trợ hàm này -> Không phải NFT)
